In [1]:
import os
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
import torchvision.models as models
from torchvision import transforms
import nibabel as nib
import random
from torch.utils.data import DataLoader
import time



final_project_path = '/home/jws2215/e6691-2024spring-project-jwss-jws2215' # vm
data_folder_path = os.path.join(final_project_path, 'BraTS2020')
train_folder_path = os.path.join(data_folder_path, 'train')
valid_folder_path = os.path.join(data_folder_path, 'valid')


if torch.cuda.is_available():
    device = torch.device('cuda')
    print("GPU name: ", torch.cuda.get_device_name(0))
    allocated_memory = torch.cuda.memory_allocated() / (1024 ** 3)  # Convert bytes to gigabytes
    cached_memory = torch.cuda.memory_reserved() / (1024 ** 3)  # Convert bytes to gigabytes
    print(f"Allocated Memory: {allocated_memory:.2f} GB")
    print(f"Cached Memory: {cached_memory:.2f} GB")
    total_memory = torch.cuda.get_device_properties(device).total_memory / (1024 ** 3)  # Convert bytes to gigabytes
    print(f"Total GPU Memory: {total_memory:.2f} GB")
else:
    print("CUDA is not available. Cannot print memory usage.")
    device = torch.device('cpu')



GPU name:  Tesla T4
Allocated Memory: 0.00 GB
Cached Memory: 0.00 GB
Total GPU Memory: 14.58 GB


In [2]:
def create_data_dictionary(folder_path):
    data_dict = {}
    subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    
    for idx, subfolder in enumerate(subfolders):
        abs_path = os.path.join(folder_path, subfolder)
        data_dict[idx] = {'absolute_path': abs_path, 'folder_name': subfolder}
    
    return data_dict

data_path_dictionary_valid = create_data_dictionary(valid_folder_path)



In [6]:
## all data is stored in (240,240,155) 

class ImageDataset(Dataset):
    def __init__(self, data_path_dictionary):
        self.data_path_dictionary = data_path_dictionary

    def __len__(self):
        # print(len(self.annotations["images"]))
        return len(self.data_path_dictionary)

    def __getitem__(self, idx):
        
        folder_name = self.data_path_dictionary[idx]["folder_name"]
        folder_path = self.data_path_dictionary[idx]["absolute_path"]
        
        seg_path = os.path.join(folder_path, folder_name + '_seg.nii')
        t1_path = os.path.join(folder_path, folder_name + '_t1.nii')
        t1ce_path = os.path.join(folder_path, folder_name + '_t1ce.nii')
        t2_path = os.path.join(folder_path, folder_name + '_t2.nii')
        flair_path = os.path.join(folder_path, folder_name + '_flair.nii')
        
        # Load .nii files as nparrays
        seg_img = nib.load(seg_path).get_fdata()
        
        t1_img = nib.load(t1_path).get_fdata() #combine these ones
        t1ce_img = nib.load(t1ce_path).get_fdata()#combine these ones
        t2_img = nib.load(t2_path).get_fdata()#combine these ones
        flair_img = nib.load(flair_path).get_fdata()#combine these ones
        
        # Combine the MRI scans into a single 4-channel image
        combined_mri = np.stack([t1_img, t1ce_img, t2_img, flair_img], axis=0)  
        
        # Convert combined_mri and seg_img to torch tensors
        # combined_mri = torch.tensor(combined_mri, dtype=torch.float32)
        # seg_img = torch.tensor(seg_img, dtype=torch.float32)
        combined_mri = torch.tensor(combined_mri, dtype=torch.int32)
        seg_img = torch.tensor(seg_img, dtype=torch.int32)

        
        #convert to binary problem:
        seg_img[seg_img != 0] = 1
        
        return combined_mri, seg_img

val_dataset = ImageDataset(data_path_dictionary_valid)
batch_size = 16  # remember each item in a batch actually contains 155 subbatches
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [7]:
from seg_models import UNet, UNetPaper
    
    
# Define the number of classes
num_output_classes = 1 
num_input_channels = 4

# Custom Models
model = UNet(num_input_channels, num_output_classes)

# Paper Models
# model = UNetPaper(num_input_channels, num_output_classes)

# Load the saved weights
saved_models_path = os.path.join(final_project_path, 'saved_models')


####### Change the name here #########
saved_model_path = os.path.join(saved_models_path, 'train_unet1_bce_lr1e-5_20e')

# Add the file name
saved_weights_path = 'best_unet.pth'
saved_weights_path = os.path.join(saved_model_path, saved_weights_path)
saved_weights = torch.load(saved_weights_path)

# Load the weights into the model
model.load_state_dict(saved_weights)
model = model.to(device)
print("model", model)

model UNet(
  (encoder_conv1): Sequential(
    (0): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (encoder_conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (encoder_conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (encoder_conv4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (bottleneck)

In [8]:
# Define functions to calculate Dice coefficient and IoU
def calculate_dice_coefficient(pred, target):
    intersection = torch.sum(pred * target)
    dice_coefficient = (2. * intersection) / (torch.sum(pred) + torch.sum(target))
    return dice_coefficient.item()

def calculate_iou(pred, target):
    intersection = torch.sum(pred * target)
    union = torch.sum(pred) + torch.sum(target) - intersection
    iou = intersection / union
    return iou.item()

In [9]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

model.eval()
dice_coefficient = 0.0
iou_score = 0.0
accuracy = 0.0
sensitivity = 0.0
specificity = 0.0
total_batches = 0

with torch.no_grad():
    for combined_mris, seg_imgs in val_loader:
        for depth_idx in range(combined_mris.size(4)):
            current_slice = combined_mris[:, :, :, :, depth_idx].to(device)
            current_seg = seg_imgs[:, :, :, depth_idx].long()
            current_seg_one_hot = current_seg.unsqueeze(1).float().to(device)

            # Predict
            outputs = model(current_slice.float())

            # Calculate Dice and IoU scores for the current depth slice
            dice_coefficient += calculate_dice_coefficient(outputs, current_seg_one_hot)
            iou_score += calculate_iou(outputs, current_seg_one_hot)

            # Flatten the predictions and ground truth for accuracy, sensitivity, and specificity
            pred_flat = torch.argmax(outputs, dim=1).view(-1)
            target_flat = current_seg.view(-1)

            # Calculate Accuracy, Sensitivity, and Specificity
            accuracy += accuracy_score(target_flat.cpu(), pred_flat.cpu())
            sensitivity += recall_score(target_flat.cpu(), pred_flat.cpu(), average='binary', pos_label=1)
            specificity += recall_score(target_flat.cpu(), pred_flat.cpu(), average='binary', pos_label=0)

            total_batches += 1

# Calculate average scores for the epoch
avg_dice_coefficient = dice_coefficient / total_batches
avg_iou_score = iou_score / total_batches
avg_accuracy = accuracy / total_batches
avg_sensitivity = sensitivity / total_batches
avg_specificity = specificity / total_batches

print("Average Dice Coefficient:", avg_dice_coefficient)
print("Average IoU Score:", avg_iou_score)
print("Average Accuracy:", avg_accuracy)
print("Average Sensitivity (Recall):", avg_sensitivity)
print("Average Specificity:", avg_specificity)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricW

Average Dice Coefficient: 0.43317471458508555
Average IoU Score: 0.35097804875495353
Average Accuracy: 0.9929263167842741
Average Sensitivity (Recall): 0.0
Average Specificity: 1.0


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
